In [1]:
from dataclasses import dataclass
import os

import torch 
from torch.utils.tensorboard import SummaryWriter
from torch_geometric.nn import RGCNConv

from custom_modules import Training, DataGenerator

2024-09-03 08:39:01.371566: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-03 08:39:01.431811: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-03 08:39:01.431862: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-03 08:39:01.431899: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-03 08:39:01.441417: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-03 08:39:01.442739: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [2]:
@dataclass
class TrainingConfig:
    output_dir = "RL_PersSched"
    num_epoch = 220000 #10'000 needs 1 hour with 3x CPU, 3GB RAM and 1x 1080ti
    max_steps = 8
    batch_size = 128
    replay_size = 10000
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    eval_every_n_epochs = 20
    lr = 1e-4
    gamma = 0.99
    hidden_dim = 3
    num_message_passing = 4
    clip_norm = 1

config = TrainingConfig()

## Training

In [3]:
tb_summary = SummaryWriter(config.output_dir, purge_step=0)
os.makedirs(config.output_dir, exist_ok=True)

In [4]:
gnn = RGCNConv(in_channels = config.hidden_dim, out_channels=config.hidden_dim, num_relations=1).to(config.device)
dim_employee = 1
projection_employees = torch.nn.Linear(dim_employee, config.hidden_dim).to(config.device)
dim_shift = 7
projection_shifts = torch.nn.Linear(dim_shift, config.hidden_dim).to(config.device)

In [12]:
len([p for p in gnn.parameters()])

3

In [6]:
optimizer = torch.optim.AdamW(
    list(gnn.parameters()) + list(projection_employees.parameters()) + list(projection_shifts.parameters()), 
    lr=config.lr, 
    amsgrad=True
    ) 

In [7]:
tb_summary.add_scalar("num_epoch", config.num_epoch, 0)
tb_summary.add_scalar("max_steps", config.max_steps, 0)
tb_summary.add_scalar("batch_size", config.batch_size, 0)
tb_summary.add_scalar("replay_size", config.replay_size, 0)
tb_summary.add_scalar("eval_every_n_epochs", config.eval_every_n_epochs, 0)
tb_summary.add_scalar("lr", config.lr, 0)
tb_summary.add_scalar("gamma", config.gamma, 0)
tb_summary.add_scalar("hidden_dim", config.hidden_dim, 0)
tb_summary.add_scalar("num_message_passing", config.num_message_passing, 0)
tb_summary.add_scalar("clip_norm", config.clip_norm, 0)
tb_summary.add_scalar("num_shifts", DataGenerator.get_week_shifts().shape[0], 0)

In [8]:
training = Training(
    gnn, 
    optimizer, 
    projection_employees,
    projection_shifts,
    tb_summary, 
    device=config.device,
    num_message_passing=config.num_message_passing, 
    gamma=config.gamma,
    max_steps=config.max_steps, 
    num_epoch=config.num_epoch, 
    batch_size=config.batch_size, 
    replay_size=config.replay_size, 
    eval_every_n_epochs=config.eval_every_n_epochs,
    output_dir=config.output_dir,
    clip_norm=config.clip_norm
) 
training.start_training()

NameError: name 'asdf' is not defined

## Save as python script

In [9]:
import os

!jupyter nbconvert --to script "RL_env.ipynb"
filename = "RL_env.py"

# delete this cell from python file
lines = []
with open(filename, 'r') as fp:
    lines = fp.readlines()
with open(filename, 'w') as fp:
    for number, line in enumerate(lines):
        if number < len(lines)-17: 
            fp.write(line)

[NbConvertApp] Converting notebook RL_env.ipynb to script
[NbConvertApp] Writing 3416 bytes to RL_env.py
